In [1]:
# Packages
import numpy as np
import os
import torch
from torchvision import models
from torchvision import datasets
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt


In [ ]:
# Data loading
dataPath = '/blue/eel6935/guzikjar/HW2/LungImages'
maskPath = 'LungMasks'

In [7]:
# Classes

class downNet1(nn.Module):
    def __init__(self):
        super(downNet1, self).__init__()

        # First 2D convolutional layer, taking in 1 input channel (image),
        # outputting 64 convolutional features, with a square kernel size of 3
        self.conv1 = nn.Conv2d(1, 64, 3, 1)

        # Activation Function
        self.act1 = nn.ReLU()

        # Second 2D convolutional layer, taking in the 64 input layers,
        # outputting 64 convolutional features, with a square kernel size of 3
        self.conv2 = nn.Conv2d(64, 64, 3, 1)

        # Activation Function
        self.act2 = nn.ReLU()
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.act2(x)
        output = x
        return output

    
class downNet2(nn.Module):
    def __init__(self):
        super(downNet2, self).__init__()
        
        self.conv1 = nn.Conv2d(64, 128, 3, 1)
        
        self.act1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(128, 128, 3, 1)
        
        self.act2 = nn.ReLU()
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.act2(x)
        output = x
        return output
        
        
class downNet3(nn.Module):
    def __init__(self):
        super(downNet3, self).__init__()
        
        self.conv1 = nn.Conv2d(128, 256, 3, 1)
        
        self.act1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(256, 256, 3, 1)
        
        self.act2 = nn.ReLU()
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.act2(x)
        output = x
        return output
        
        
class downNet4(nn.Module):
    def __init__(self):
        super(downNet4, self).__init__()
        
        self.conv1 = nn.Conv2d(256, 512, 3, 1)
        
        self.act1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(512, 512, 3, 1)
        
        self.act2 = nn.ReLU()
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.act2(x)
        output = x
        return output
        
class upNet1(nn.Module):
    def __init__(self):
        super(upNet1, self).__init__()

        self.conv1 = nn.Conv2d(1024, 512, 3, 1)

        self.act1 = nn.ReLU()

        self.conv2 = nn.Conv2d(512, 512, 3, 1)

        self.act2 = nn.ReLU()
        
        self.upSamp = nn.ConvTranspose2d(512, 256, 2, 2)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.act2(x)
        x = self.upSamp(x)
        output = x
        return output

    
class upNet2(nn.Module):
    def __init__(self):
        super(upNet2, self).__init__()
        
        self.conv1 = nn.Conv2d(512, 256, 3, 1)
        
        self.act1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(256, 256, 3, 1)
        
        self.act2 = nn.ReLU()
        
        self.upSamp = nn.ConvTranspose2d(256, 128, 2, 2)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.act2(x)
        x = self.upSamp(x)
        output = x
        return output
        
        
class upNet3(nn.Module):
    def __init__(self):
        super(upNet3, self).__init__()
        
        self.conv1 = nn.Conv2d(256, 128, 3, 1)
        
        self.act1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(128, 128, 3, 1)
        
        self.act2 = nn.ReLU()
        
        self.upSamp = nn.ConvTranspose2d(128, 64, 2, 2)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.act2(x)
        x = self.upSamp(x)
        output = x
        return output
        
        
class upNet4(nn.Module):
    def __init__(self):
        super(upNet4, self).__init__()
        
        self.conv1 = nn.Conv2d(128, 64, 3, 1)
        
        self.act1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(64, 64, 3, 1)
        
        self.act2 = nn.ReLU()
        
        # 2 classes, so final output should be 2
        self.conv3 = nn.Conv2d(64, 2, 1, 1) 
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.act2(x)
        x = self.conv3(x)
        output = x
        return output
        
        
class bottleNeck(nn.Module):
    def __init__(self):
        super(bottleNeck, self).__init__()
        
        self.conv1 = nn.Conv2d(512, 1024, 3, 1)
        
        self.act1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(1024, 1024, 3, 1)
        
        self.act2 = nn.ReLU()
        
        self.upSamp = nn.ConvTranspose2d(1024, 512, 2, 2)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.act2(x)
        x = self.upSamp(x)
        output = x
        return output
        
     
class pool(nn.Module):
    def __init__(self):
        super(pool, self).__init__()

        self.pooling = nn.MaxPool2d(2, 2)
        
    def forward(self, x):
        x = self.pooling(x)
        output = x
        return output
        
        
        

my_nn = upNet4()
print(my_nn)

upNet4(
  (conv1): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
  (act1): ReLU()
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (act2): ReLU()
  (conv3): Conv2d(64, 2, kernel_size=(1, 1), stride=(1, 1))
)
